
# <div style="text-align: right"> Decision Trees from scratch. </div>

---

<div style="text-align: right"> Geoff Counihan - Oct 6, 2017 </div>

### Notes

---

Unclear what the difference between sklearn's implementation and mine is, again. There must be some default arguments that are being set.
    
__Additions__: Entropy, Test cases, Random Forest

In [1]:
from sklearn.datasets import load_iris

In [4]:
import numpy as np
import operator

In [5]:
iris = load_iris()
X = iris.data[:,:2]
y = iris.target

In [6]:
Xy = np.column_stack((X,y))
Xy_point = Xy[:3]
print(Xy_point)

[[ 5.1  3.5  0. ]
 [ 4.9  3.   0. ]
 [ 4.7  3.2  0. ]]


### Similarity

---

__Numeric__ data needs a similarity metric like euclidean or manhattan. 

__Categorical__ data needs a similarity metric like gini or entropy

__Gini__ - is a measure of purity of split. basically, it looks at the class composition in the resulting two groups and quantifies between 0 and .5 in a two class problem. lower is better.

In [7]:
def gini_score(groups,classes):
    n_samples = sum([len(group) for group in groups])
    gini = 0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        #print(size)
        for class_val in classes:
            #print(group.shape)
            p = (group[:,-1] == class_val).sum() / size
            score += p * p
        gini += (1.0 - score) * (size / n_samples)
        #print(gini)
    return gini

In [8]:
# test cases
z = 40
groups = [Xy[:z,:],Xy[z:,:]]
#groups = [Xy[np.where(Xy[:,-1] == i)[0]] for i in classes]

classes = np.unique(Xy[:,-1])

In [9]:
gini_score(groups,classes)

0.42424242424242425

__Split feature__ - splits a feature into two groups based on value

In [10]:
def split(feat, val, Xy):
    Xi_left = np.array([]).reshape(0,3)
    Xi_right = np.array([]).reshape(0,3)
    for i in Xy:
        #print(i.shape)
        if i[feat] <= val:
            Xi_left = np.vstack((Xi_left,i))
        if i[feat] > val:
            Xi_right = np.vstack((Xi_right,i))
    return Xi_left, Xi_right

In [11]:
a, b = split(0, 6.3, Xy)

__Best split__ - produces a split feature based on the best information gain

In [12]:
def best_split(Xy):
    classes = np.unique(Xy[:,-1])
    best_feat = 999
    best_val = 999
    best_score = 999
    best_groups = None
    for feat in range(Xy.shape[1]-1):
        for i in Xy:
            groups = split(feat, i[feat], Xy)
            #print(groups)
            gini = gini_score(groups, classes)
            #print('feat {}, valued < {}, scored {}'.format(feat,i[feat], gini))
            if gini < best_score:
                best_feat = feat
                best_val = i[feat]
                best_score = gini
                best_groups = groups
    output = {}
    output['feat'] = best_feat
    output['val'] = best_val
    output['groups'] = best_groups
    return output

In [13]:
split_dict = best_split(Xy)

__Terminal nodes__ - we need to make a decision when to stop building a tree. This usually means taking arguments __max tree depth__ and __min num in node__

In [14]:
def terminal_node(group):
    classes, counts = np.unique(group[:,-1],return_counts=True)
    return classes[np.argmax(counts)]

In [15]:
z = 40
groups = [Xy[:z,:],Xy[z:,:]]
group = groups[0]
terminal_node(group)

0.0

__Split branch__ - recursively split data again and again with arguments for early stops

In [16]:
def split_branch(node, max_depth, min_num_sample, depth):
    left_node, right_node = node['groups']
    del(node['groups'])
    if not isinstance(left_node,np.ndarray) or not isinstance(right_node,np.ndarray):
        node['left'] = node['right'] = terminal_node(left_node + right_node)
        return
    if depth >= max_depth:
        node['left'] = terminal_node(left_node)
        node['right'] = terminal_node(right_node)
        return
    if len(left_node) <= min_num_sample:
        node['left'] = terminal_node(left_node)
    else:
        node['left'] = best_split(left_node)
        split_branch(node['left'], max_depth, min_num_sample, depth+1)
    if len(right_node) <= min_num_sample:
        node['right'] = terminal_node(right_node)
    else:
        node['right'] = best_split(right_node)
        split_branch(node['right'], max_depth, min_num_sample, depth+1)

In [17]:
split_dict = best_split(Xy)
left, right = split_dict['groups']

__Build tree__ - the simple function to take inputs and then build the tree

In [18]:
def build_tree(Xy, max_depth, min_num_sample):
    root = best_split(Xy)
    split_branch(root, max_depth, min_num_sample, 1)
    return root

In [19]:
Xy = np.column_stack((X,y))
tree = build_tree(Xy, 2, 30)

__Display tree__ - primative way of displaying the splits the tree makes

In [20]:
def display_tree(node, depth=0):
    if isinstance(node,dict):
        print('{}[feat{} < {:.2f}]'.format(depth*'\t',(node['feat']+1), node['val']))
        display_tree(node['left'], depth+1)
        display_tree(node['right'], depth+1)
    else:
        print('{}[{}]'.format(depth*'\t', node))
        

In [21]:
display_tree(tree)

[feat1 < 5.40]
	[feat2 < 2.70]
		[1.0]
		[0.0]
	[feat1 < 6.10]
		[1.0]
		[2.0]


__Predict sample__ - make a recursive function to travel down the tree structure to determine class of new sample

In [22]:
def predict_sample(node,sample):
    print(node)
    if sample[node['feat']] < node['val']:
        if isinstance(node['left'],dict):
            return predict_sample(node['left'],sample)
        else:
            return node['left']
    else:
        if isinstance(node['right'],dict):
            return predict_sample(node['right'],sample)
        else:
            return node['right']

In [23]:
sample = Xy[0]
sample

array([ 5.1,  3.5,  0. ])

In [24]:
predict_sample(tree,sample)

{'feat': 0, 'val': 5.4000000000000004, 'left': {'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}, 'right': {'feat': 0, 'val': 6.0999999999999996, 'left': 1.0, 'right': 2.0}}
{'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}


0.0

__Predict__ - predict a set of samples

In [25]:
def predict(X):
    y_pred = np.array([])
    for i in X:
        y_pred = np.append(y_pred,predict_sample(tree,i))
    return y_pred

In [26]:
y_pred = predict(X)

{'feat': 0, 'val': 5.4000000000000004, 'left': {'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}, 'right': {'feat': 0, 'val': 6.0999999999999996, 'left': 1.0, 'right': 2.0}}
{'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}
{'feat': 0, 'val': 5.4000000000000004, 'left': {'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}, 'right': {'feat': 0, 'val': 6.0999999999999996, 'left': 1.0, 'right': 2.0}}
{'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}
{'feat': 0, 'val': 5.4000000000000004, 'left': {'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}, 'right': {'feat': 0, 'val': 6.0999999999999996, 'left': 1.0, 'right': 2.0}}
{'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}
{'feat': 0, 'val': 5.4000000000000004, 'left': {'feat': 1, 'val': 2.7000000000000002, 'left': 1.0, 'right': 0.0}, 'right': {'feat': 0, 'val': 6.0999999999999996, 'left': 1.0, 'right': 2.0}}
{'feat': 1, 'val': 2.7000000000000002, 'le

In [27]:
y_pred

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,
        2.,  1.,  2.,  1.,  2.,  1.,  2.,  0.,  1.,  1.,  1.,  2.,  1.,
        2.,  1.,  1.,  2.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,  1.,
        2.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  2.,  1.,  2.,  2.,
        2.,  2.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  1.,  1.,  2.,  2.,
        2.,  2.,  1.,  2.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  1.,  2.,  2.,  2.,  1.,
        2.,  2.,  2.,  2.,  2.,  2.,  1.])

### Create class.

---

__Tie__ - Added to modify behavior when there is a tie for majority class

In [28]:
class decision_tree(object):
    def __init__(self, max_depth, min_num_split):
        self.max_depth = max_depth
        self.min_num_sample = min_num_split
    
    def gini(self, X):
        if np.min(X) < 0:
            X -= np.amin(X)
        #X = X.flatten()
        X += .00001
        X = np.sort(X)
        i = np.arange(1,X.shape[0]+1)
        n = X.shape[0]
        return ((np.sum((2 * i - n - 1) * X)) / (n * np.sum(X)))
    
    def gini_score(self, groups, classes):
        n_samples = sum([len(group) for group in groups])
        gini = 0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue
            score = 0.0
            #print(size)
            for class_val in classes:
                #print(group.shape)
                p = (group[:,-1] == class_val).sum() / size
                #print(p)
                score += p * p
            gini += (1.0 - score) * (size / n_samples)
            #print(gini)
        return gini
    
    def split(self, feat, val, Xy):
#         Xi_left = np.array([]).reshape(0,3)
#         Xi_right = np.array([]).reshape(0,3)
        Xi_left = np.array([]).reshape(0,self.Xy.shape[1])
        Xi_right = np.array([]).reshape(0,self.Xy.shape[1])
        for i in Xy:
            #print(i.shape)
            if i[feat] <= val:
                Xi_left = np.vstack((Xi_left,i))
            if i[feat] > val:
                Xi_right = np.vstack((Xi_right,i))
        return Xi_left, Xi_right
    
    def best_split(self, Xy):
        classes = np.unique(Xy[:,-1])
        best_feat = 999
        best_val = 999
        best_score = 999
        best_groups = None
        for feat in range(Xy.shape[1]-1):
            for i in Xy:
                groups = self.split(feat, i[feat], Xy)
                #print(groups)
                gini = self.gini_score(groups, classes)
                #print('feat {}, valued < {}, scored {}'.format(feat,i[feat], gini))
                if gini < best_score:
                    best_feat = feat
                    best_val = i[feat]
                    best_score = gini
                    best_groups = groups
        output = {}
        output['feat'] = best_feat
        output['val'] = best_val
        output['groups'] = best_groups
        return output
    
    def terminal_node(self, group):
        # errored out: couldn't np.unique(nothing) or something - doesn't happen all the time
        #print(group[:,-1])
        classes, counts = np.unique(group[:,-1],return_counts=True)
        return classes[np.argmax(counts)]
            
    def split_branch(self, node, depth):
        left_node, right_node = node['groups']
        del(node['groups'])
        if not isinstance(left_node,np.ndarray) or not isinstance(right_node,np.ndarray):
            node['left'] = node['right'] = self.terminal_node(left_node + right_node)
            return
        if depth >= self.max_depth:
            node['left'] = self.terminal_node(left_node)
            node['right'] = self.terminal_node(right_node)
            return
        if len(left_node) <= self.min_num_sample:
            node['left'] = self.terminal_node(left_node)
        else:
            node['left'] = self.best_split(left_node)
            self.split_branch(node['left'], depth+1)
        if len(right_node) <= self.min_num_sample:
            node['right'] = self.terminal_node(right_node)
        else:
            node['right'] = self.best_split(right_node)
            self.split_branch(node['right'], depth+1)
    
    def build_tree(self):
        '''Recursively build tree, unclear if this is the correct way
        
        '''
        
        self.root = self.best_split(self.Xy)
        #print(self.root)
        self.split_branch(self.root, 1) # i don't understand how this is working, pointed to node?
        #print(self.root)
        return self.root
    
    def fit(self,X,y):
        '''Save training data.
        
        '''
        self.X = X
        self.y = y
        self.Xy = np.column_stack((X, y))
        self.build_tree()

    def display_tree(self, depth=0):
        if isinstance(self.root,dict):
            print('{}[feat{} < {:.2f}]'.format(depth*'\t',(self.root['feat']+1), self.root['val']))
            display_tree(self.root['left'], depth+1)
            display_tree(self.root['right'], depth+1)
        else:
            print('{}[{}]'.format(depth*'\t', self.root))
            
    def predict_sample(self, node, sample):
        #print(node)
        if sample[node['feat']] < node['val']:
            if isinstance(node['left'],dict):
                return self.predict_sample(node['left'],sample)
            else:
                return node['left']
        else:
            if isinstance(node['right'],dict):
                return self.predict_sample(node['right'],sample)
            else:
                return node['right']
    
    def predict(self, X_test):
        self.y_pred = np.array([])
        for i in X_test:
            #print(i)
            self.y_pred = np.append(self.y_pred,self.predict_sample(self.root,i))
        return self.y_pred
        

### Test.

---

In [29]:
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris.data[:,:2]
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=35)

In [30]:
dt = decision_tree(max_depth=2,min_num_split=30)

In [31]:
dt.fit(X_train,y_train)
#dt.fit(X,y)

In [32]:
%%time
my_pred = dt.predict(X_test)

CPU times: user 964 µs, sys: 933 µs, total: 1.9 ms
Wall time: 1.25 ms


### Display tree.
---

In [33]:
dt.display_tree()

[feat1 < 5.40]
	[feat2 < 2.70]
		[1.0]
		[0.0]
	[feat1 < 6.20]
		[1.0]
		[2.0]


### Compare performance

---

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [35]:
clf = DecisionTreeClassifier(max_depth=2,min_samples_split=30)

In [36]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=30, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [37]:
sk_pred = clf.predict(X_test)

### Sample points

---

In [38]:
a = X_test[:3]

In [39]:
print('sk_pred: {}'.format(clf.predict(a)))
print('my_pred: {}'.format(dt.predict(a)))
print('true: {}'.format(y_test[:3]))

sk_pred: [1 1 2]
my_pred: [ 1.  1.  2.]
true: [1 1 2]


### Accuracy differences

---

I'm unclear how sklearn differs. Will need to look deeper

In [40]:
def accuracy(pred,true):
    correct = 0
    pred_len = len(pred)
    for i in range(pred_len):
        if pred[i] == true[i]:
            correct += 1
    return correct/pred_len

In [41]:
accuracy(my_pred,y_test)

0.7631578947368421

In [42]:
accuracy(sk_pred,y_test)

0.7894736842105263

In [43]:
list(zip(my_pred,sk_pred,y_test))

[(1.0, 1, 1),
 (1.0, 1, 1),
 (2.0, 2, 2),
 (2.0, 2, 1),
 (0.0, 0, 0),
 (2.0, 2, 2),
 (2.0, 2, 2),
 (1.0, 1, 1),
 (1.0, 1, 1),
 (0.0, 0, 0),
 (1.0, 0, 1),
 (2.0, 2, 2),
 (0.0, 0, 0),
 (2.0, 2, 2),
 (0.0, 0, 0),
 (1.0, 1, 2),
 (1.0, 1, 1),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (2.0, 2, 1),
 (1.0, 1, 1),
 (2.0, 2, 2),
 (2.0, 2, 1),
 (0.0, 0, 0),
 (1.0, 0, 0),
 (1.0, 0, 0),
 (2.0, 2, 2),
 (0.0, 0, 0),
 (2.0, 2, 2),
 (0.0, 0, 0),
 (1.0, 1, 1),
 (1.0, 1, 2),
 (0.0, 0, 0),
 (2.0, 2, 1),
 (1.0, 1, 2),
 (0.0, 0, 0),
 (2.0, 2, 2)]